<a id='1.1'></a>
## Install all the packages through FinRL library


In [1]:
## install finrl library
# !pip install git+https://github.com/amritgos/FinRL.git

In [2]:
# !git clone -q https://github.com/amritgos/FinRL.git

In [3]:
# cd FinRL


<a id='1.2'></a>
## Packages Required 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## Importing Packages

In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv

from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts

import sys
sys.path.append("../FinRL")

In [6]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Downloading Data


In [11]:
print(config.SENSEX_30_TICKER)

['ASIANPAINT.BO' 'AXISBANK.BO' 'BAJAJ-AUTO.BO' 'BAJFINANCE.BO'
 'BAJAJFINSV.BO' 'BHARTIARTL.BO' 'DRREDDY.BO' 'HCLTECH.BO' 'HDFC.BO'
 'HDFCBANK.BO' 'HINDUNILVR.BO' 'ICICIBANK.BO' 'INDUSINDBK.BO' 'INFY.BO'
 'ITC.BO' 'KOTAKBANK.BO' 'LT.BO' 'M&M.BO' 'MARUTI.BO' 'NESTLEIND.BO'
 'NTPC.BO' 'ONGC.BO' 'POWERGRID.BO' 'RELIANCE.BO' 'SBIN.BO' 'SUNPHARMA.BO'
 'TCS.BO' 'TECHM.BO' 'TITAN.BO' 'ULTRACEMCO.BO']


In [30]:
try:
    df = pd.read_csv('datasets/Price_data.csv')
except:
    df = YahooDownloader(start_date = '2016-01-01',
                        end_date = '2021-01-01',
                        ticker_list = config.SENSEX_30_TICKER).fetch_data()
    df.to_csv('datasets/Price_data.csv', index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [29]:
df.head()

,Unnamed: 0,date,open,high,low,close,volume,tic,day
0,0,2016-01-01,882.950012,885.500000,878.500000,847.758850,17377.0,ASIANPAINT.BO,4
1,1,2016-01-01,450.000000,452.700012,445.799988,440.748108,433052.0,AXISBANK.BO,4
2,2,2016-01-01,2530.000000,2530.000000,2512.149902,2137.924316,9055.0,BAJAJ-AUTO.BO,4
3,3,2016-01-01,1990.000000,2010.000000,1960.500000,1980.499756,6305.0,BAJAJFINSV.BO,4
4,4,2016-01-01,602.255005,609.994995,600.590027,600.510315,10350.0,BAJFINANCE.BO,4


In [16]:
df.shape

(36960, 8)

# Preprocessing Data


In [17]:
# vix_df = pd.read_csv('datasets/IndiaVIX_data.csv')

In [21]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    use_vix=False,
                    vix_df=None,
                    user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [22]:
df.shape

(36960, 25)

In [23]:
df.head()

,date,open,high,low,close,volume,tic,day,macd_x,boll_ub_x,...,close_60_sma_x,turbulence,macd_y,boll_ub_y,boll_lb_y,rsi_30_y,cci_30_y,dx_30_y,close_30_sma_y,close_60_sma_y
0,2016-01-01,882.950012,885.500000,878.500000,847.758850,17377.0,ASIANPAINT.BO,4,0.0,848.680556,...,847.758850,0.0,0.0,848.680556,847.31865,100.0,66.666667,100.0,847.758850,847.758850
1232,2016-01-01,450.000000,452.700012,445.799988,440.748108,433052.0,AXISBANK.BO,4,0.0,848.680556,...,440.748108,0.0,0.0,848.680556,847.31865,100.0,66.666667,100.0,440.748108,440.748108
2464,2016-01-01,2530.000000,2530.000000,2512.149902,2137.924316,9055.0,BAJAJ-AUTO.BO,4,0.0,848.680556,...,2137.924316,0.0,0.0,848.680556,847.31865,100.0,66.666667,100.0,2137.924316,2137.924316
3696,2016-01-01,1990.000000,2010.000000,1960.500000,1980.499756,6305.0,BAJAJFINSV.BO,4,0.0,848.680556,...,1980.499756,0.0,0.0,848.680556,847.31865,100.0,66.666667,100.0,1980.499756,1980.499756
4928,2016-01-01,602.255005,609.994995,600.590027,600.510315,10350.0,BAJFINANCE.BO,4,0.0,848.680556,...,600.510315,0.0,0.0,848.680556,847.31865,100.0,66.666667,100.0,600.510315,600.510315


<a id='4'></a>
# Setting up Environment


## Training data split: 2009-01-01 to 2018-12-31

In [24]:
train = data_split(df, '2016-01-01','2019-07-01')
#trade = data_split(df, '2020-01-01', config.END_DATE)

In [25]:
train.head()

,date,open,high,low,close,volume,tic,day,macd_x,boll_ub_x,...,close_60_sma_x,turbulence,macd_y,boll_ub_y,boll_lb_y,rsi_30_y,cci_30_y,dx_30_y,close_30_sma_y,close_60_sma_y
0,2016-01-01,882.950012,885.500000,878.500000,847.758850,17377.0,ASIANPAINT.BO,4,0.0,848.680556,...,847.758850,0.0,0.0,848.680556,847.31865,100.0,66.666667,100.0,847.758850,847.758850
0,2016-01-01,450.000000,452.700012,445.799988,440.748108,433052.0,AXISBANK.BO,4,0.0,848.680556,...,440.748108,0.0,0.0,848.680556,847.31865,100.0,66.666667,100.0,440.748108,440.748108
0,2016-01-01,2530.000000,2530.000000,2512.149902,2137.924316,9055.0,BAJAJ-AUTO.BO,4,0.0,848.680556,...,2137.924316,0.0,0.0,848.680556,847.31865,100.0,66.666667,100.0,2137.924316,2137.924316
0,2016-01-01,1990.000000,2010.000000,1960.500000,1980.499756,6305.0,BAJAJFINSV.BO,4,0.0,848.680556,...,1980.499756,0.0,0.0,848.680556,847.31865,100.0,66.666667,100.0,1980.499756,1980.499756
0,2016-01-01,602.255005,609.994995,600.590027,600.510315,10350.0,BAJFINANCE.BO,4,0.0,848.680556,...,600.510315,0.0,0.0,848.680556,847.31865,100.0,66.666667,100.0,600.510315,600.510315


## Environment for Portfolio Optimization


In [26]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 30, State Space: 301


In [27]:
env_kwargs = {
    "hmax": 1000, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "turbulence_threshold": None,
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

KeyError: 'macd'

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# A2C Algorithm

In [ ]:
# initialize
agent = DRLAgent(env = env_train)

In [ ]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.001}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.001}
Using cuda device


In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=60000)

Logging to tensorboard_log/a2c/a2c_1
------------------------------------
| time/                 |          |
|    fps                | 80       |
|    iterations         | 100      |
|    time_elapsed       | 6        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -42.4    |
|    explained_variance | 0        |
|    learning_rate      | 0.001    |
|    n_updates          | 99       |
|    policy_loss        | -78.5    |
|    std                | 0.996    |
|    value_loss         | 29.2     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 91       |
|    iterations         | 200      |
|    time_elapsed       | 10       |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -42.4    |
|    explained_variance | 0        |
|    learning_rate      | 0.001    |
|    n_updates          | 199      |
|

## Trading
Assume that we have Rs10,00,000 initial capital at 2019-01-01. We use the A2C model to trade SENSEX stocks.

In [ ]:
data_turbulence = processed[(df.date<'2019-01-01') & (df.date>='2009-01-01')]
insample_turbulence = data_turbulence.drop_duplicates(subset=['date'])

In [ ]:
turbulence_threshold = np.quantile(insample_turbulence.turbulence.values,0.95)

In [ ]:
data_vix = processed[(df.date<'2019-01-01') & (df.date>='2009-01-01')]
insample_vix = data_turbulence.drop_duplicates(subset=['date'])

In [ ]:
vix_threshold = np.quantile(insample_vix.VIX.values,0.95)

In [ ]:
env_kwargs = {
    "hmax": 1000, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "turbulence_threshold": turbulence_threshold,
    "vix_threshold": vix_threshold,
    "reward_scaling": 1e-4
    
}

In [ ]:
trade = data_split(df,'2019-01-01', '2021-01-01')
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)

In [ ]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

In [ ]:
df_actions.to_csv('df_actions.csv')

<a id='6'></a>
# Backtesting methodology

<a id='6.1'></a>
## BackTestStats


In [ ]:
from pyfolio import timeseries
s1 = get_daily_return(df_daily_return, value_col_name="account_value")
df_daily_return['daily_return'] = s1.values
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")

In [ ]:
print("==============DRL Strategy Stats===========")
perf_stats_all

<a id='6.2'></a>
## BackTestPlot

In [ ]:
import pyfolio
%matplotlib inline

baseline_df = get_baseline(
        ticker='^BSESN', start='2019-01-01', end='2021-01-01'
    )

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=baseline_returns, set_context=False)